In [1]:
# This document details the circuit design decisions made for the LTC7871 buck converter. The goal here is to provide an easy
# way to evaluate how component values will affect the overall behavior and performance, without having to restrict myself to
# an excel sheet or a SPICE simulation. This is a living document and will be updated as I learn more about the LTC7871 and
# develop the hardware.